In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

Since in this assignment, we are only working on English, spacy for english only is needed.

In [5]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [6]:
spacy_en = spacy.load('en_core_web_sm')

In [7]:
def tokenize_n(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [8]:
QUE = Field(tokenize = tokenize_n,
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

ANS = Field(tokenize = tokenize_n, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
#STR1 = Field(sequential = True, tokenize = tokenize_n, init_token='<sos>',eos_token='<eos>', lower=True)
#STR2 = Field(sequential = True, tokenize = tokenize_n, init_token='<sos>',eos_token='<eos>', lower=True)

In [9]:
fields = [('que', QUE),('ans',ANS)]

In [11]:
import pandas as pd
from google.colab import files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Overview of the Question Answer Dataset

The file "question_answer_pairs.txt" contains the questions and answers. The first line of the file contains 
column names for the tab-separated data fields in the file. This first line follows:

ArticleTitle    Question        Answer  DifficultyFromQuestioner        DifficultyFromAnswerer  ArticleFile

Field 1 is the name of the Wikipedia article from which questions and answers initially came.
Field 2 is the question.
Field 3 is the answer.
Field 4 is the prescribed difficulty rating for the question as given to the question-writer. 
Field 5 is a difficulty rating assigned by the individual who evaluated and answered the question, 
which may differ from the difficulty in field 4.

Since we are only focussing on the text part of the dataset, which is question and answer, I am only picking up these two columns.

In [12]:
df=pd.read_csv('/content/drive/MyDrive/dataset/cmu/S08/question_answer_pairs.txt',sep='\t',
               header=0,lineterminator='\n',dtype=pd.StringDtype(),encoding = "ISO-8859-1",
               names= ['ArticleTitle','Question','Answer','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'],usecols = ['Question','Answer'])

In [ ]:
for col in df.columns:
    print(f'\t column {col} | {type(df[col][0])}')

	 column Question | <class 'str'>
	 column Answer | <class 'str'>


In [19]:
df.shape

(2539, 2)

Some Examples from the dataset

In [24]:
df.head()

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
1,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Did Lincoln sign the National Banking Act of 1...,yes
3,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Did his mother die of pneumonia?,no


In [14]:
df1=pd.read_csv('/content/drive/MyDrive/dataset/cmu/S09/question_answer_pairs.txt',sep='\t',
               header=0,lineterminator='\n',dtype=pd.StringDtype(),encoding = "ISO-8859-1",
               names= ['ArticleTitle','Question','Answer','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'],usecols = ['Question','Answer'])

In [18]:
df=df.append(df1)

In [20]:
df2=pd.read_csv('/content/drive/MyDrive/dataset/cmu/S10/question_answer_pairs.txt',sep='\t',
               header=0,lineterminator='\n',dtype=pd.StringDtype(),encoding = "ISO-8859-1",
               names= ['ArticleTitle','Question','Answer','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'],usecols = ['Question','Answer'])

In [21]:
df=df.append(df2)

In [22]:
df.shape

(3997, 2)

In [23]:
df=df.reset_index(drop=True)

Lets first create the DataFrame

In [25]:
example = [torchtext.legacy.data.Example.fromlist([str(df.Question[i]),str(df.Answer[i])], fields) for i in range(df.shape[0])]

# New Section

Now create the dataset

In [26]:
from torchtext.legacy import data

In [27]:
# Creating dataset
QADataset = data.Dataset(example, fields)

Finally, we can split into training  and test sets by using the split() method:

In [28]:
(train_set, test_set) = QADataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [ ]:
print(f"Number of training examples: {len(train_set.examples)}")
print(f"Number of testing examples: {len(test_set.examples)}")

In [ ]:
print(vars(train_set.examples[0]))

{'que': ['where', 'was', 'michael', 'faraday', 'born', '?'], 'ans': ['michael', 'faraday', 'was', 'born', 'in', 'newington', 'butts', '.']}


In [30]:
QUE.build_vocab(train_set, min_freq = 2)
ANS.build_vocab(train_set, min_freq = 2)

In [31]:
print(f"Unique tokens in source  vocabulary: {len(QUE.vocab)}")
print(f"Unique tokens in target vocabulary: {len(ANS.vocab)}")

Unique tokens in source  vocabulary: 2215
Unique tokens in target vocabulary: 1293


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
BATCH_SIZE = 128

train_iterator,  test_iterator = BucketIterator.splits(
    (train_set, test_set), sort_key = lambda x: len(x.que),
    sort_within_batch=True,
    batch_size = BATCH_SIZE, 
    device = device)

In [34]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [35]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [37]:
INPUT_DIM = len(QUE.vocab)
OUTPUT_DIM = len(ANS.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [38]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2215, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1293, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1293, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,917,773 trainable parameters


In [40]:
optimizer = optim.Adam(model.parameters())

In [41]:
TRG_PAD_IDX = ANS.vocab.stoi[ANS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
b=next(iter(train_iterator))

In [ ]:
src,trg=b.que,b.ans

In [ ]:
src.shape

torch.Size([8, 128])

In [ ]:
len(src[0])

128

In [ ]:
o=model(src,trg)

In [ ]:
o.shape

torch.Size([39, 128, 646])

In [ ]:
top1 = o.argmax(1)

In [ ]:
top1

tensor([[  0,   0,   0,  ...,   0,   0,   0],
        [102,  30, 115,  ...,  28, 116,  91],
        [ 55,  13,  54,  ...,  48,  95,  53],
        ...,
        [ 37,  35,  21,  ...,  35,  22,  95],
        [105,  35,  69,  ...,  35,  65,  52],
        [ 94,  19,  30,  ...,  51,  42,  30]])

In [ ]:
top1.shape

torch.Size([39, 646])

In [ ]:
 ou_dim = o.shape[-1]

In [ ]:
ou_dim

646

In [ ]:
output = o[1:].view(-1, ou_dim)
tr = trg[1:].view(-1)

In [ ]:
o[1:].shape

torch.Size([38, 128, 646])

In [ ]:
output.shape, tr.shape

(torch.Size([4864, 646]), torch.Size([4864]))

In [ ]:
 optimizer.zero_grad()
 loss = criterion(output, tr)

In [ ]:
loss.item()

6.470804214477539

In [ ]:
     
loss.backward()


In [ ]:
clip=1      
torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
optimizer.step()

In [42]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.que
        trg = batch.ans
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.que
            trg = batch.ans

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
#The name VALID_LOSS actually indicated TEST_LOSS. There was no validation data. The model was evaluated on Test Data.
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 8s
	Train Loss: 5.026 | Train PPL: 152.376
	 Val. Loss: 3.706 |  Val. PPL:  40.675
Epoch: 02 | Time: 1m 6s
	Train Loss: 4.261 | Train PPL:  70.895
	 Val. Loss: 3.517 |  Val. PPL:  33.697
Epoch: 03 | Time: 1m 7s
	Train Loss: 4.130 | Train PPL:  62.156
	 Val. Loss: 3.539 |  Val. PPL:  34.422
Epoch: 04 | Time: 1m 4s
	Train Loss: 4.076 | Train PPL:  58.906
	 Val. Loss: 3.517 |  Val. PPL:  33.680
Epoch: 05 | Time: 1m 7s
	Train Loss: 3.939 | Train PPL:  51.386
	 Val. Loss: 3.428 |  Val. PPL:  30.826
Epoch: 06 | Time: 1m 6s
	Train Loss: 3.859 | Train PPL:  47.405
	 Val. Loss: 3.473 |  Val. PPL:  32.232
Epoch: 07 | Time: 1m 7s
	Train Loss: 3.786 | Train PPL:  44.086
	 Val. Loss: 3.412 |  Val. PPL:  30.318
Epoch: 08 | Time: 1m 6s
	Train Loss: 3.726 | Train PPL:  41.519
	 Val. Loss: 3.439 |  Val. PPL:  31.168
Epoch: 09 | Time: 1m 6s
	Train Loss: 3.691 | Train PPL:  40.094
	 Val. Loss: 3.538 |  Val. PPL:  34.401
Epoch: 10 | Time: 1m 8s
	Train Loss: 3.674 | Train PPL:  39.429


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.221 | Test PPL:  25.062 |
